In [ ]:
# INITIALIZATION:  initialize subpopulations to cover different parts of the search space
# MIGRATION: let it occur randomly. individuals are chosen at random to migrate to a random destination
#             OR tournament selection can be used
#             OR something like proportional selection, where "immigrants" are accepted based on the proportion
#             of its fitness compared to that of the destination island



In [1]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import math
import random

from copy import deepcopy

# Create chromosome blueprint, initialize the population, and create a way to display
# the population

class Chromosome:
    
    def __init__(self, genes = None):
        
        if(genes == None):
            x = np.random.randint(-10, 10)
            y = np.random.randint(-10, 10)
            
        else:
            x = genes["x"]
            y = genes["y"]
    
        #self.genes = {"x": x}
        self.genes = {"x": x, "y": y}
        
        self.fitness = None
        
def create_population(num_chromosomes):
    
    population = []
    for i in range(num_chromosomes):
        population.append(Chromosome())
    return population 

def display_population(population, string):
    
    print(f"\nPopulation after {string}:\n")
    for i, chromosome in enumerate(population):
        print("Chromosome %s : x = %s, y = %s, fitness = %s" 
              % (i, chromosome.genes["x"], chromosome.genes["y"], chromosome.fitness))

# maps a chromosome representation into a scalar value

def fitness(x, y):
    
    #x^2
    #return (x ** 2)
    
    #Matyas Function
    return (x ** 2 + y ** 2) * 0.26 - (x * y) * 0.48
    
    #Himmelblau's Function
    #return (x ** 2 + y - 11) ** 2 + (x + y ** 2 - 7) ** 2
    
    #Rastrigin function
#     a_n = np.asarray(range(1,21))
#     n= len(a_n)
#     for a in a_n:
#         sum += x 
    
    #Ackley function
    #return( -20 * np.exp(-0.2 * np.sqrt(0.5 * (x ** 2 + y ** 2))) - np.exp(0.5*(np.cos(2*np.pi*x) + np.cos(2*np.pi*y))) - np.e + 20)
# Create function to update each chromosome's fitness

def evaluate_population(population):

    for chromosome in population:
        chromosome.fitness = fitness(chromosome.genes["x"], chromosome.genes["y"])
        
    scores = [chromosome.fitness for chromosome in population]
    
    indices = np.argsort(scores) #this sorts each row from greatest to least and creates a list of indices
    
    return list(np.asarray(population)[indices])

# Create selection function(s). this is the function that chooses parents to reproduce.

# Max num selected would be something to experiment with / improve on! 

def roulette_wheel(population, p_num_selected, epsilon = 1e-4, max_num_selected = 4):
    
    sum_of_fitnesses = calculate_sum_of_fitnesses(population)
    
    # calculate relative fitness. Roulette wheel made
    
    try:
        scores = [1 - (chromo.fitness / (sum_of_fitnesses + epsilon)) for chromo in population]
    except:
        #print("LOOK AT ME: ", sum_of_fitnesses)
        from IPython import embed
        embed()
        #exit()
        
    # spin the created roulet wheel --> P selected chromos
    
    selected = []
    
    for i in range(p_num_selected):
        
        # If parent is already selected, pick another? 
        
        while(1):
            
            sigma = random.uniform(0, np.max(scores))

            total = 0
            for j in range(len(scores)):
                total = total + scores[j]
                if(total >= sigma):
                    break

            the_chosen = population[j]
            
            #print("Chosen: x = %s, y = %s" % (the_chosen.genes["x"], the_chosen.genes["y"]))

            # get number of times chosen has appeared in selected parents

            total = 0
            for chromosome in selected:
                if(the_chosen == chromosome):
                    total = total + 1
            
            if(total < max_num_selected):
                break  
            else:
                scores.pop(j)
                population.pop(j)
                 
        selected.append(the_chosen)
        #print("CHOSEN IS ADDED")

    return selected 

def calculate_sum_of_fitnesses(population):
    sum_of_fitnesses = 0
    for chromosome in population:
        sum_of_fitnesses += chromosome.fitness
    return sum_of_fitnesses

def create_matches(parent_group, req_num_parents):
    
    num_groups = int(len(parent_group) / req_num_parents)
    
    # get sets of subgroups that are going to produce one child
    
    couples = []
        
    for i in range(num_groups):
        
        # first get the indices of parents for each subgroup (this would be something to improve on :-))
        
        indices = [ np.random.randint(0, len(parent_group)) 
                    for j in range(req_num_parents) ]
        
        # populate subgroup with chromosomes 
        
        group = [parent_group[index] for index in indices]
        
        couples.append(group)
    
    return couples

def selection(population, elite_percent, req_num_parents = 2):
    
    parents = []
    
    if elite_percent > 0:
        
        elite_group = []
        
        num_chromosomes = len(population)
        e_num_selected = math.ceil(elite_percent * num_chromosomes) 
        
        # most elite chromos automatically get into next generation
        
        for i in range(e_num_selected):
            elite_group.append(population[i])
    
    else:
        elite_group = []
        e_num_selected = 0
        
    # let's build up the rest of the parents list using roulette wheel.
   
    num_chromosomes = len(population)
    p_num_selected = int((num_chromosomes - e_num_selected) * req_num_parents)
    
    parent_group = roulette_wheel(population, p_num_selected)
    
    parent_group = create_matches(parent_group, req_num_parents)

    return elite_group, parent_group

# Create simplex cross over function(s)
# you can choose to have two parents, three parents, k parents
# if you do k-point, stick with two parents
###### i really don't know what kind of crossover i did here ####

def calculate_mean(population):
    
    num_genes = len(population[-1].genes.keys())
    
    mean = np.zeros(num_genes)
    for chromosome in population:
        genes = chromosome.genes
        for i, current_gene in enumerate(genes.keys()):
            mean[i] += chromosome.genes[current_gene]
        
    return mean / len(population)
        
def simplex_crossover(parent_groups):
    
    children = []
    for parents in parent_groups:
    
        # select random parent for simplex equation 
        
        index = np.random.randint(0, len(parents))
        rand_chromo = parents[index]
        
        # get all possible genes
        
        all_genes = parents[-1].genes.keys()
        
        # run simplex on each gene
        
        simplexed_genes = {}
        for current_gene in all_genes:
            
            epsilon = np.random.rand()
            info = [chromo.genes[current_gene] for chromo in parents] 
            
            # This would be an area to improve!! Make better selection for rand chromo.
            
            agg = math.ceil(np.mean(info) + (rand_chromo.genes[current_gene] - np.mean(info)) * epsilon)
            
            simplexed_genes[current_gene] = agg
        
        # create child from simplexed genes
        
        children.append(Chromosome(simplexed_genes))
    
    return children

# implement mutation here

def mutation(children, mutate_chance, mutate_scale = 0.5):
    
    kiddos = []
    
    for child in children:
        
        alpha = np.random.rand()
        
        # mutate if random allows 
        
        if(alpha <= mutate_chance):
            
            # mutate all child genes
            
            all_genes = child.genes.keys()
        
            for current_gene in all_genes:
                flip = -1 if(np.random.rand() >= 0.5) else 1
                offset = flip * child.genes[current_gene] * mutate_scale
                child.genes[current_gene] = math.ceil(child.genes[current_gene] + offset)
            
        kiddos.append(child)
    
    return kiddos 

def mutation_constant(population, generation):
    
    # the strategy is to decrease sigma as the number of generations increases so we get 
    # small variations near the optimum, preventing individuals from jumping over the 
    # minimum
    
    if generation >= num_generations * (3/4):
            sigma = 0.05
    elif generation < num_generations * (3/4) and generation >= num_generations / 2:
            sigma = 0.1
    elif generation < num_generations / 2 and generation >= num_generations / 4:
            sigma = 0.15
    elif generation < num_generations / 4:
            sigma = 0.2
            
    # each individual is mutated by adding a gaussian random value
    gaussian = random.uniform(0,sigma)
    # can I make this value hover around zero?
    #gaussian = random.uniform(-sigma,sigma)
    
    for chromosome in population:
        chromosome.genes["x"] = chromosome.genes["x"] + gaussian
        
    # evaluate fitness before returning
    evaluate_population(population)
    
    return population

# stats methods
def get_min(population):
    
    fitness = []
    for chromosome in population:
        fitness.append(chromosome.fitness)
    return min(fitness)
    
def get_max(population):
    fitness = []
    for chromosome in population:
        fitness.append(chromosome.fitness)
    return max(fitness)

def get_mean(population):
    fitness = []
    sum = 0
    for chromosome in population:
        sum += chromosome.fitness
    return sum / len(population)

In [ ]:
def run_generation(initial_pop):
    
    pop = evaluate_population(initial_pop)
    
    # Selection for children and optional elitism 
    
    elite_group, parent_group = selection(deepcopy(pop), elite_percent)
    
    # Create some kids, via cross-over / mutation 
    
    children = simplex_crossover(parent_group)
    
    #display_population(children, "children")
    
    children = mutation(children, mutate_chance)
    
    #display_population(children,"mutation")
    
    # Update population for new generation 
    
    pop = elite_group + children
    
    return pop


# create some params #
    
num_chromosomes = 20
num_generations = 50
elite_percent = 0
mutate_chance = 0.4
display_target = 5

num_islands = 3

# Create island population

all_islands = []

for i in range(num_islands):
    
    island = {}
    island["fitness_min"] = []
    island["fitness_max"] = []
    island["fitness_avg"] = []
    island["pop"] = create_population(num_chromosomes)
    
    all_islands.append(island)

# Train Island GA

for i in range(num_generations):
    
    # Train each island ONCE. 
    
    for j in range(num_islands):
        
        # current island is a population of a GA :)
            
        current_island = all_islands[j]
        
        if(i == 0):
            current_island["orig_pop"] = deepcopy(current_island["pop"])
        
        current_island["pop"] = run_generation(current_island["pop"])
        #current_island["fitness_min"].append(get_min(current_island["pop"]))
        #current_island["fitness_max"].append(get_max(current_island["pop"]))
        #current_island["fitness_avg"].append(get_mean(current_island["pop"]))

    
    from IPython import embed
    embed()

Python 3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.0.1 ]
Type 'copyright', 'credits' or 'license' for more information
IPython 8.4.0 -- An enhanced Interactive Python. Type '?' for help.

